In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("D:\\ml_ve\\Dataset\\intents.json").read()
intents = json.loads(data_file)
# print(intents)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(classes)
print(documents)

['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']
[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How

In [3]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# print(words)

In [4]:
classes = sorted(list(set(classes)))
print (len(documents), "documents",'\n')
print (len(classes), "classes", classes,'\n')
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents 

9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks'] 

88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [5]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-5-c5f30ffaef46>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [6]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

C:\Users\nisil\AppData\Roaming\Python\Python38\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 3s 6ms/step - loss: 2.2336 - accuracy: 0.1064
Epoch 2/200
10/10 [==============================] - 0s 6ms/step - loss: 2.1727 - accuracy: 0.1489
Epoch 3/200
10/10 [==============================] - 0s 5ms/step - loss: 2.0639 - accuracy: 0.2979
Epoch 4/200
10/10 [==============================] - 0s 5ms/step - loss: 2.0161 - accuracy: 0.2553
Epoch 5/200
10/10 [==============================] - 0s 7ms/step - loss: 1.8870 - accuracy: 0.4255
Epoch 6/200
10/10 [==============================] - 0s 12ms/step - loss: 1.7839 - accuracy: 0.4681
Epoch 7/200
10/10 [==============================] - 0s 11ms/step - loss: 1.6618 - accuracy: 0.4894
Epoch 8/200
10/10 [==============================] - 0s 9ms/step - loss: 1.4884 - accuracy: 0.6596
Epoch 9/200
10/10 [==============================] - 0s 8ms/step - loss: 1.2480 - accuracy: 0.6809
Epoch 10/200
10/10 [==============================] - 0s 6ms/step - loss: 1.2640 - accuracy: 0.5957
Epoch 1

10/10 [==============================] - 0s 9ms/step - loss: 0.0442 - accuracy: 1.0000
Epoch 83/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0402 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0941 - accuracy: 0.9787
Epoch 86/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0827 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0313 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0806 - accuracy: 0.9787
Epoch 89/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0933 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0762 - accuracy: 0.9787
Epoch 91/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0717 - accuracy: 0.9787
Epoch 92/200


10/10 [==============================] - 0s 5ms/step - loss: 0.0529 - accuracy: 0.9787
Epoch 164/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0187 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0312 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0234 - accuracy: 0.9787
Epoch 168/200
10/10 [==============================] - 0s 9ms/step - loss: 0.0680 - accuracy: 0.9787
Epoch 169/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0260 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0394 - accuracy: 0.9787
Epoch 172/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0280 - accuracy: 1.0000
Epoc

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open("D:\\ml_ve\\Dataset\\intents.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [9]:
# Creating GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()